In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [ ]:
# Loading the dataset

data = pd.read_csv('Churn_Modelling.csv')
data.head()

Pre-Processing

In [99]:
# Preprocessing the data

data= data.drop(['RowNumber','CustomerId', 'Surname'],axis=1)


In [ ]:
data

In [ ]:
#Encode Categorical Variables

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder


In [ ]:
# Assigning geo_encoder values to column names 'Geography_France', 'Geography_Germany', 'Geography_Spain'

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns= onehot_encoder_geo.get_feature_names_out(['Geography']) )
geo_encoded_df

In [ ]:
# Combine onehot encoded columns with the original data
data = pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data.head()

In [105]:
#Save the LabelEncoder, OneHotEncoder and StandardScalar in pickle for later use during inference

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)



In [ ]:
data.head()

In [107]:
#Divide the dataset into dependent and independent features

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# #Scale the features

scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [108]:
with open('scalar.pkl','wb') as file:
    pickle.dump(scaler,file)

Artificial Neural Network - Implementation (ANN)

In [109]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [ ]:
# Building ANN model
model = Sequential([
    Dense(64,activation='relu',kernel_regularizer=l2(0.01),input_shape = (X_train.shape[1],)), # Hidden Layer 1 Connected with Input Layer #12 features
    Dropout(0.3),
    Dense(32, activation='relu',kernel_regularizer=l2(0.01)), # Hidden Layer 2
    Dropout(0.3),
    Dense(1,activation='sigmoid') # Output Layer
])

In [ ]:
model.summary()

In [112]:
#Compile Model
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001)
loss = tensorflow.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [113]:
# Setting up the TensorBoard
log_dir ="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [114]:
# Setting up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=50,restore_best_weights=True)

In [ ]:
#Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data = (X_test,y_test),
    epochs = 100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

In [ ]:
# Save the model file

model.save('model.h5')
    

In [ ]:
#Load TensorBoard Extenstion
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit20250111-195139